# HyperOpt

In [2]:

import pandas as pd
import xgboost as xgb

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split as tts

In [17]:
df=pd.read_csv('data/diamonds_train.csv')
df=df.dropna()

df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95


In [18]:
X=df.drop('price', axis=1)
y=df.price

In [8]:
Xpd.get_dummies(X)

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,62.4,58.0,6.83,6.79,4.25,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
1,0.32,63.0,57.0,4.35,4.38,2.75,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.71,65.5,55.0,5.62,5.53,3.65,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.41,63.8,56.0,4.68,4.72,3.00,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.02,60.5,59.0,6.55,6.51,3.95,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,62.7,57.0,7.10,7.04,4.43,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
40451,2.02,57.1,60.0,8.31,8.25,4.73,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
40452,1.01,62.7,56.0,6.37,6.42,4.01,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
40453,0.33,61.9,54.3,4.45,4.47,2.76,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [19]:
X.clarity.unique()

array(['VS2', 'VS1', 'SI1', 'SI2', 'IF', 'VVS1', 'VVS2', 'I1'],
      dtype=object)

In [20]:
clarity={'I1':0, 'SI2':1, 'SI1':2, 'VS2':3, 'VS1':4,
         'VVS2':5, 'VVS1':6, 'IF':7}


cut={'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}


color={'J':0, 'I':1, 'H':2, 'G':3, 'F':4, 'E':5, 'D':6}

In [21]:
def label(s,dic):
    return dic[s]

In [22]:
X.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [23]:
X['clarity'] = X['clarity'].apply(lambda x: label(x,clarity))


In [27]:
X['cut'] = X['cut'].apply(lambda x: label(x,cut))
X['color'] = X['color'].apply(lambda x: label(x,color))

In [28]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.21,3,0,3,62.4,58.0,6.83,6.79,4.25
1,0.32,2,2,3,63.0,57.0,4.35,4.38,2.75
2,0.71,0,3,4,65.5,55.0,5.62,5.53,3.65
3,0.41,1,6,2,63.8,56.0,4.68,4.72,3.00
4,1.02,4,3,2,60.5,59.0,6.55,6.51,3.95
...,...,...,...,...,...,...,...,...,...
40450,1.34,4,3,4,62.7,57.0,7.10,7.04,4.43
40451,2.02,1,4,1,57.1,60.0,8.31,8.25,4.73
40452,1.01,4,2,2,62.7,56.0,6.37,6.42,4.01
40453,0.33,4,0,4,61.9,54.3,4.45,4.47,2.76


In [29]:
X_train, X_test, y_train, y_test = tts(X, y)

,carat,cut,color,clarity,depth,table,x,y,z
9874,0.61,2,0,1,58.1,63.0,5.62,5.60,3.26
21915,0.32,4,4,5,60.7,57.0,4.42,4.45,2.69
30241,0.55,4,5,3,59.3,57.0,5.43,5.39,3.21
27631,0.33,4,2,5,61.3,56.0,4.43,4.47,2.73
10987,0.71,4,4,1,60.7,57.0,5.78,5.81,3.52
...,...,...,...,...,...,...,...,...,...
35719,1.20,4,2,4,62.1,57.0,6.77,6.82,4.22
24188,1.20,2,3,2,61.9,58.0,6.74,6.79,4.19
35274,1.50,3,3,2,61.2,58.0,7.34,7.44,4.52
30067,0.43,2,1,5,62.8,58.0,4.80,4.82,3.02


In [32]:
space = {
    'n_estimators': hp.quniform('n_estimators',10, 1000,25),
    
    'learning_rate': hp.uniform('learning_rate', 0.0001, 1.0),
    
    'max_depth': hp.quniform('x_max_depth', 4,16,1),
    
    'min_child_weight':hp.quniform('x_min_child', 1,10,1),
    
    'subsample': hp.uniform('x_subsample', 0.7,1),
    
     'gamma':hp.uniform('x_gamma', 0.1, 0.5),
    
    'reg_lambda':hp.uniform('x_reg_lambda', 0, 1),
}

In [34]:
def objetivo(x):
    
    modelo = xgb.XGBRegressor(
            n_estimators = int(x['n_estimators']),
            learning_rate = x['learning_rate'],
            max_depth = int(x['max_depth']),
            min_child_weight = x['min_child_weight'],
            subsample = x['subsample'],
            gamma = x['gamma'],
            reg_lambda = x['reg_lambda'],
            objective = 'reg:squarederror'
            )
    eval_set = [(X_train, y_train), (X_test,y_test)]
    
    modelo.fit(X_train, y_train, eval_set=eval_set, eval_metric='rmse', verbose=False) #Entrenamiento es fit()
    
    y_pred=modelo.predict(X_test) #Prediccion
    
    rmse=mse(y_test, y_pred)**0.5
    
    return {'loss': rmse, 'status':STATUS_OK}

In [36]:
trials_reg=Trials()

In [37]:
best=fmin(fn=objetivo, space=space, algo=tpe.suggest, max_evals=10, trials=Trials())


100%|█████████████████████████████████████████████████| 10/10 [01:50<00:00, 11.04s/trial, best loss: 517.6086573161734]


In [38]:
best

{'learning_rate': 0.028054792393677246,
 'n_estimators': 550.0,
 'x_gamma': 0.3860734852616452,
 'x_max_depth': 6.0,
 'x_min_child': 8.0,
 'x_reg_lambda': 0.8714043372996012,
 'x_subsample': 0.7579282549649675}

In [39]:
m=xgb.XGBRegressor()
m.fit(X_train, y_train)
y_pred=m.predict(X_test)
mse(y_test, y_pred)**0.5


541.6157225269491